# Наследование

** Наследование ** — механизм языка, позволяющий описать новый класс на основе уже существующего (родительского, базового) класса.
* Класс-потомок может добавить собственные методы и свойства, а также пользоваться родительскими методами и свойствами.
* Позволяет строить иерархии классов.
* Является одним из основных принципов объектно-ориентированного
программирования.

В версиях до 2.2 некоторые объектно-ориентированные возможности Python были заметно ограничены. Начиная с версии 2.2, объектная система Python была существенно переработана и дополнена.
* В целях совместимости с существующим кодом в Python 2 существуют две системы типов: классы нового типа (new-style classes) и классы старого типа (old-style classes, classic classes).
* Для создания класса нового типа следует унаследовать его от любого другого класса нового типа. Все стандартные классы являются классами нового типа. Базовым из них является класс object.
* Если в Python 2 не указывать базовый класс или унаследовать его от другого класса старого типа, то данный класс является классом старого типа.

_Классы старого типа нужны только для обратной совместимости. В новом коде следует использовать *только классы нового типа*._

### В Python 3 все классы являются классами нового типа и наследуются по умолчанию от object.

In [3]:
class Base():
    def __init__(self):
        self.greeting = "Hello"
        
    def method(self):
        print(self.greeting)
        
class Inh(Base):
    pass

class OtherInh(Base):
    def __init__(self, greeting):
        self.greeting = greeting
        
        
class AndOtherInh(Base):
    def __init__(self, greeting):
        self.greeting = greeting

    def method(self, name):
        print('{0} and {1}!'.format(self.greeting, name))

if __name__ == '__main__':
    inh = Inh()
    inh.method()
    
    other_inh = OtherInh("Hello world!")
    other_inh.method()
    
    and_other_inh = AndOtherInh("Hello world ")
    and_other_inh.method("Evgen")

Hello
Hello world!
Hello world  and Evgen!


In [4]:
class Figure(object):
    def __init__(self, side):
        self.side = side


class Square(Figure):
    def draw(self):
        for i in range(self.side):
            print('*' * self.side)


class Triangle(Figure):
    def draw(self):
        for i in range(self.side):
            print('*' * i)


if __name__ == '__main__':
    square = Square(10)
    square.draw()

    print()

    triangle = Triangle(5)
    triangle.draw()

**********
**********
**********
**********
**********
**********
**********
**********
**********
**********


*
**
***
****


При множественном наследовании у класса может быть более одного предка. В этом случае класс наследует методы всех предков. Достоинство такого подхода в большей гибкости, однако он может быть потенциальным источником ошибок.

Список базовых классов указывается через запятую в круглых скобках после имени данного класса:
```
   class Pegasus(Horse, Bird):
       pass
```

In [5]:
"""
Пример множественного наследования
"""

class Bird(object):
    def fly(self):
        print('I am flying.')


class Horse(object):
    def run(self):
        print('I am running.')


class Pegasus(Horse, Bird):
    pass


def main():
    bird = Bird()
    horse = Horse()
    pegasus = Pegasus()

    bird.fly()
    # bird.run()  # ошибка
    print()

    # horse.fly()  # ошибка
    horse.run()
    print()

    pegasus.fly()
    pegasus.run()


if __name__ == '__main__':
    main()

I am flying.

I am running.

I am flying.
I am running.


### Method Resolution Order
https://en.wikipedia.org/wiki/C3_linearization

In [12]:
"""
Недостатки такого подхода:
 - усложняется рефакторинг и поддержка кода (хотя эта проблема
   решается путём использования таких IDE, как PyCharm);
 - логика кода жёстко привязана к иерархии наследования классов
   и подвержена ошибкам, особенно при использовании множественного
   наследования.
"""

class Base:
    attr = 'Base attribute'

    def method(self):
        print('Base method, current class is', self.__class__.__name__)


class Child(Base):
    attr = 'Redefined attribute'

    @staticmethod
    def get_superclass_attr():
        return Base.attr  # получение атрибута класса Base

    
    def get_super(self):
        return super().attr
    
    def method(self):  # переопределение метода
        Base.method(self)  # вызов метода суперкласса
        print('Child method, current class is', self.__class__.__name__)


def main():
    print('Base:')
    print(Base.attr)
    base_instance = Base()
    base_instance.method()

    print()

    print('Child:')
    print(Child.attr)
    print(Child.get_superclass_attr())
    child_instance = Child()
    child_instance.get_super()


if __name__ == '__main__':
    main()

Base:
Base attribute
Base method, current class is Base

Child:
Redefined attribute
Base attribute


Для классов нового типа доступен другой способ, решающий описанные ранее
проблемы. Существует специальный класс super, экземпляры которого являются
специальными прокси-объектами, привязанными к данной иерархии классов
и предоставляющими доступ к атрибутам следующего класса в линеаризации
того класса, в котором был создан объект super.

Таким образом, при помощи super можно получить доступ к атрибутам
и методам суперкласса, не называя его имени, причём это будет давать
корректные результаты даже при использовании множественного наследования.

В Python 2 в качестве параметров конструктора super передаются текующий
класс и экземпляр текущего класса. super ищет класс-предок в линеаризации
указанного класса и все обращения к атрибутам созданного объекта отображаются
на найденный класс, причём все методы привязаны к указанному объекту-экземпляру.

Пример: ``` super(MyClass, self).method() ```

В Python 3 данные параметры можно не указывать. В таком случае будут
автоматически получены текущий класс и экземпляр. То есть, вызов

``` super().method() ```

эквивалентен

``` super(__class__, self).method() ```

In [13]:
class Base(object):
    attr = 'Base attribute'

    def method(self):
        print('Base method, current class is', self.__class__.__name__)


class Child(Base):
    attr = 'Redefined attribute'

    def get_superclass_attr(self):
        return super().attr  # получение атрибута класса Base

    def method(self):  # переопределение метода
        super().method()  # вызов метода суперкласса
        print('Child method, current class is', self.__class__.__name__)


if __name__ == '__main__':
    print('Base:')
    print(Base.attr)
    base_instance = Base()
    base_instance.method()

    print()

    print('Child:')
    print(Child.attr)
    child_instance = Child()
    print(child_instance.get_superclass_attr())
    child_instance.method()

    print()    

Base:
Base attribute
Base method, current class is Base

Child:
Redefined attribute
Base attribute
Base method, current class is Child
Child method, current class is Child



В большинстве языков программирования, в которых есть конструкция,
аналогичная super(), она может вызвана только внутри соответсвующего
метода подкласса, причём часто только в качестве его первого оператора
(впрочем, это наиболее типичный сценарий её использования).
В Python, как "побочный эффект" того, как работает его реализация super,
её можно вызвать где угодно, даже за пределами класса.

In [14]:
super(Child, child_instance).method()

Base method, current class is Child


In [24]:
# -*- coding: utf-8 -*-

"""
Пример использования super при множественном наследовании
"""


class Animal(object):
    def __init__(self):
        self.can_fly = False
        self.can_run = False

    def print_abilities(self):
        print(self.__class__.__name__)
        print('Can fly:', self.can_fly)
        print('Can run:', self.can_run)
        print()


class Bird(Animal):
    def __init__(self):
        super().__init__()
        self.can_fly = True


class Horse(Animal):
    def __init__(self):
        super().__init__()
        self.can_run = True


class Pegasus(Horse, Bird):
    pass


def main():
    bird = Bird()
    bird.print_abilities()

    horse = Horse()
    horse.print_abilities()

    pegasus = Pegasus()
    pegasus.print_abilities()


if __name__ == '__main__':
    main()

Bird
Can fly: True
Can run: False

Horse
Can fly: False
Can run: True

Pegasus
Can fly: True
Can run: True



** isinstance(obj, cls) ** проверяет, является ли obj экземпляром класса cls
или класса, который является наследником класса cls

In [25]:
print(isinstance(8, int))  # True
print(isinstance('str', int))  # False
print(isinstance(True, bool))  # True
print(isinstance(True, int))  # True, так как bool -- подкласс int
print(isinstance('a string', object))  # True, всё является объектами
print(isinstance(None, object))  # True, даже None
print(isinstance(False, str))  # False
print(isinstance(int, type))  # True, любой класс -- объект-экземпляр метакласса type
print(isinstance(42, type))  # False, 42 -- это не тип данных

True
False
True
True
True
True
False
True
False


** issubclass(cls, base) ** проверяет, является ли класс cls наследником класса base.

In [16]:
print(issubclass(bool, int))  # True
print(issubclass(float, int))  # False
print(issubclass(int, float))  # False
print(issubclass(complex, type))  # False
print(issubclass(complex, object))  # True, всё наследуется от object

class Base(object):
    pass

class Child(Base):
    pass

print(issubclass(Child, Base))  # True
print(issubclass(Base, object))  # True
print(issubclass(Child, object))  # True по транзитивности

True
False
False
False
True
True
True
True
